In [2]:
import pandas as pd
dataset = pd.read_csv("/content/train.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134486 entries, 0 to 134485
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   ID                         134486 non-null  int64 
 1   TargetID                   134486 non-null  object
 2   DRUGID                     134486 non-null  object
 3   DRUGTYPE                   134486 non-null  object
 4   Drug_high_status           134486 non-null  object
 5   DRUGNAME                   134486 non-null  object
 6   PUBCHCID                   134486 non-null  int64 
 7   Disease_of_highest_status  134486 non-null  object
 8   Drug_Status                134486 non-null  object
 9   UNIPROID                   134486 non-null  object
 10  TARGNAME                   134486 non-null  object
 11  GENENAME                   134486 non-null  object
 12  SYNONYMS                   134486 non-null  object
 13  FUNCTION                   134486 non-null  

In [3]:
input_cols =['DRUGTYPE','Drug_high_status', 'DRUGNAME', 'Disease_of_highest_status','Drug_Status','GENENAME','BIOCLASS','SEQUENCE', 'Disease']
target_cols=['Target_Status']

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(dataset['Target_Status'])

dataset['Target_Status'] = label_encoder.transform(dataset['Target_Status'])

In [5]:
dataset['Target_Status']

,Target_Status
0,31
1,1
2,1
3,22
4,1
...,...
134481,22
134482,18
134483,14
134484,26


In [6]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(dataset[input_cols])
encoded_cols = encoder.transform(dataset[input_cols])
encoded_cols

<134486x5268 sparse matrix of type '<class 'numpy.float64'>'
	with 1210374 stored elements in Compressed Sparse Row format>

In [36]:
import xgboost as xgb
from sklearn.metrics import f1_score, accuracy_score
xgb_model = xgb.XGBClassifier(n_estimators=1000, learning_rate=0.25, random_state=42,)
xgb_model.fit(encoded_cols, dataset[target_cols])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.25, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [59]:
test_data =pd.read_csv("/content/test (1).csv")
test_data.head()

,ID,TargetID,DRUGID,DRUGTYPE,Drug_high_status,DRUGNAME,PUBCHCID,Disease_of_highest_status,Drug_Status,UNIPROID,TARGNAME,GENENAME,SYNONYMS,FUNCTION,BIOCLASS,SEQUENCE,Disease,Accession Number
0,1076,T51487,D0HO2T,Small molecular drug,Phase 2,AZD7325,23581869,Anxiety disorder,Phase 2,GBRA1_HUMAN,GABA(A) receptor alpha-1 (GABRA1),GABRA1,Gamma-aminobutyric acid receptor subunit alpha...,Ligand-gated chloride channel which is a compo...,Ligand-gated ion channel,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,Malaria,P14867
1,190816,T74312,D0Q4EQ,Small molecular drug,Investigative,JNJ-38158471,16742063,Angiogenesis disorder,Investigative,FLT3_HUMAN,Fms-like tyrosine kinase 3 (FLT-3),FLT3,Stem cell tyrosine kinase 1; STK1; STK-1; Rece...,Tyrosine-protein kinase that acts as cell-surf...,Kinase,MPALARDGGQLPLLVVFSAMIFGTITNQDLPVIKCVLINHKNNDSS...,Malignant haematopoietic neoplasm,P36888
2,180551,T40474,D0R0MW,Small molecular drug,Approved,Sunitinib,5329102,Gastrointestinal stromal tumour,Approved,MET_HUMAN,Proto-oncogene c-Met (MET),MET,Tyrosine-protein kinase Met; Scatter factor re...,Regulates many physiological processes includi...,Kinase,MKAPAVLAPGILVLLFTLVQRSNGECKEALAKSEMNVNMKYQLPNF...,Liver cancer,Q86W50
3,51630,T67162,D0T2XU,Small molecular drug,Approved,Promethazine,4927,Nausea,Approved,DRD2_HUMAN,Dopamine D2 receptor (D2R),DRD2,Dopamine receptor 2; D(2) dopamine receptor,Dopamine receptor whose activity is mediated b...,GPCR rhodopsin,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...,Depression,P14416
4,50566,T67162,D0H7KF,Small molecular drug,Approved,Quetiapine,5002,Alcohol dependence,Phase 3,DRD2_HUMAN,Dopamine D2 receptor (D2R),DRD2,Dopamine receptor 2; D(2) dopamine receptor,Dopamine receptor whose activity is mediated b...,GPCR rhodopsin,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...,Lymphoma,P14416


In [60]:
input_test_cols = ['DRUGTYPE','Drug_high_status', 'DRUGNAME', 'Disease_of_highest_status','Drug_Status','GENENAME','BIOCLASS','SEQUENCE', 'Disease']

In [67]:
encoded_test_cols = encoder.transform(test_data[input_test_cols])

In [69]:
xgb_predictions = xgb_model.predict(encoded_test_cols)

In [70]:
true_preds = label_encoder.inverse_transform(xgb_predictions)

In [71]:
true_preds[1]

'Phase 3'

In [73]:
predictions_df = pd.DataFrame({'ID': test_data['ID'], 'Prediction': true_preds})

In [74]:
predictions_df.head()

,ID,Prediction
0,1076,Approved
1,190816,Phase 3
2,180551,Phase 3
3,51630,Approved
4,50566,Phase 1/2


In [75]:
predictions_df.to_csv('predictions.csv', index=False)

In [68]:
print("Shape of training data:", encoded_cols.shape)
print("Shape of test data:", encoded_test_cols.shape)


Shape of training data: (134486, 5268)
Shape of test data: (58114, 5268)


In [77]:
print(test_data[dataset.duplicated(['ID'])])

Empty DataFrame
Columns: [ID, TargetID, DRUGID, DRUGTYPE, Drug_high_status, DRUGNAME, PUBCHCID, Disease_of_highest_status, Drug_Status, UNIPROID, TARGNAME, GENENAME, SYNONYMS, FUNCTION, BIOCLASS, SEQUENCE, Disease, Accession Number]
Index: []


<ipython-input-77-829b49d55727>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(test_data[dataset.duplicated(['ID'])])


In [104]:
sumbit_data = pd.read_csv("/content/predictions.csv")

In [105]:
print(sumbit_data[sumbit_data.duplicated(['ID'])])

           ID               Prediction
29382  173243                 Approved
29383   87098                 Approved
29384  102940                  Phase 1
29385   77413                 Approved
29386   58346                 Approved
...       ...                      ...
29854  166753                  Phase 1
29855   32949                  Phase 2
29856  102862            Investigative
29857  185715  Discontinued in Phase 3
29858  126738  Discontinued in Phase 3

[477 rows x 2 columns]


In [106]:
sumbit_data.drop_duplicates(subset=['ID'], keep='first', inplace = True)

In [108]:
sumbit_data

,ID,Prediction
0,1076,Approved
1,190816,Phase 3
2,180551,Phase 3
3,51630,Approved
4,50566,Phase 1/2
...,...,...
58109,33517,Phase 2
58110,177422,Phase 1/2
58111,520,Approved
58112,15371,Investigative


In [109]:
sumbit_data.to_csv('submission.csv', index=False)